In [1]:
import sys
import os
import pickle
import math


sys.path.append("/Users/Bya/git/predictEPL/config/")
sys.path.append("/Users/Bya/git/predictEPL/utils/")


import paths
import names
import tokenizer
import useful_methods

### Step 1: Data, Game Infos

In [2]:
# Game Infos
os.chdir(paths.READ_PATH_GAME_INFO)
dfGameInfos = useful_methods.csv_dic_df('game_infos.csv')

# Convert number strings to integers
dfGameInfos['GW'] = [int(GW) for GW in dfGameInfos['GW']]
dfGameInfos['score_ht_home'] = [int(number) for number in dfGameInfos['score_ht_home']]
dfGameInfos['score_ht_away'] = [int(number) for number in dfGameInfos['score_ht_away']]
dfGameInfos['score_ft_home'] = [int(number) for number in dfGameInfos['score_ft_home']]
dfGameInfos['score_ft_away'] = [int(number) for number in dfGameInfos['score_ft_away']]

dfGameInfos.head()

,GW,away_team,date,home_team,score_ft_away,score_ft_home,score_ht_away,score_ht_home,time
0,1,Tottenham,Sat. 8 Aug.,United,0,1,0,1,11:45
1,1,Watford,Sat. 8 Aug.,Everton,2,2,1,0,14:00
2,1,Sunderland,Sat. 8 Aug.,Leicester,2,4,0,3,14:00
3,1,Crystal,Sat. 8 Aug.,Norwich,3,1,1,0,14:00
4,1,Villa,Sat. 8 Aug.,Bournemouth,1,0,0,0,14:00


In [3]:
weeks = list(dfGameInfos['GW'])
home_teams = list(dfGameInfos['home_team'])
away_teams = list(dfGameInfos['away_team'])
score_ht_homes = list(dfGameInfos['score_ht_home'])
score_ht_aways = list(dfGameInfos['score_ht_away'])
score_ft_homes = list(dfGameInfos['score_ft_home']) 
score_ft_aways = list(dfGameInfos['score_ft_away'])

### Step 2: Load Naive Bayes Classifier

In [5]:
# Read Naive Bayes Classifier
os.chdir("/Users/Bya/Dropbox/Research/datas/Detecter/")
with open('nb_tweets_sentiment_detector(hash_emolex).pkl', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'utf-8'
    nb_detector_reloaded = u.load()

classifier = nb_detector_reloaded

### Step 3: Classify All Games by Naive Bayes

In [6]:
# Tweet Classifier
# Return POS, NEG %
def Classify(dfFilter, side, start=1, end=63):
    
    # set time intervals
    df = dfFilter[(dfFilter['side'] == side) &
                  (dfFilter['ith_minute'] >= start) & (dfFilter['ith_minute'] <= end)].copy()
    
    # Classify teach text and save scores
    texts = list(df['text'])
    nb_negs, nb_poss = [], []

    for text in texts:
        # classify text
        nb_neg, nb_pos = classifier.predict_proba([text])[0] # scoring here

        # save results
        nb_negs.append(nb_neg)
        nb_poss.append(nb_pos)
    
    
    # Sum all scores
    nb_pos_sum = sum(nb_poss)
    nb_neg_sum = sum(nb_negs)
    
    # Return POS, NEG %
    nb_pos_perc = nb_pos_sum / (nb_pos_sum + nb_neg_sum)
    nb_neg_perc = nb_neg_sum / (nb_pos_sum + nb_neg_sum)
    
    return nb_pos_perc, nb_neg_perc

In [7]:
# Get Single Game Datas
def EplClassifyAll():
    # results container
    res_ht_nb = []
    
    # run through each games
    for index in range(len(dfGameInfos)):
        
        try:
            # Read Game as DF
            week = weeks[index]
            team_home = home_teams[index]
            team_away = away_teams[index]

            os.chdir(paths.READ_PATH_EXTRACTED_CSV + 'GW' + str(week) + '/SingleGames')
            df = useful_methods.csv_dic_df(team_home + "_vs_" + team_away + ".csv")

            # Filtering. Removing bots
            dfFilter = useful_methods.FilterDF(df)
            dfFilter['ith_minute'] = [int(ith_minute) for ith_minute in dfFilter['ith_minute']]

            # Classify Texts
            nb_pos_home, nb_neg_home = Classify(dfFilter, 'home')
            nb_pos_away, nb_neg_away = Classify(dfFilter, 'away')
            print(week, nb_pos_home, nb_neg_home, nb_pos_away, nb_neg_away)
            
            # Append Result
            res_ht_nb.append([index, nb_pos_home, nb_neg_home, nb_pos_away, nb_neg_away])
        
        except:
            continue
            
    return res_ht_nb

In [8]:
%time res_epl_nb = EplClassifyAll()

4 0.281951473032 0.718048526968 0.399604416547 0.600395583453
4 0.429328290034 0.570671709966 0.330078286637 0.669921713363
4 0.35381079511 0.64618920489 0.349925328887 0.650074671113
4 0.430403923132 0.569596076868 0.592774412415 0.407225587585
4 0.386479406427 0.613520593573 0.474675349015 0.525324650985
4 0.590976835992 0.409023164008 0.431176197796 0.568823802204
4 0.501528518763 0.498471481237 0.393871874608 0.606128125392
4 0.390110590762 0.609889409238 0.344599351934 0.655400648066
4 0.38499034455 0.61500965545 0.334013790683 0.665986209317
5 0.302149577771 0.697850422229 0.286534539745 0.713465460255
5 0.566130112555 0.433869887445 0.579655799637 0.420344200363
5 0.370004730803 0.629995269197 0.44440640656 0.55559359344
5 0.32293943457 0.67706056543 0.37029743465 0.62970256535
5 0.418935414186 0.581064585814 0.456169818454 0.543830181546
5 0.540944474004 0.459055525996 0.438841721939 0.561158278061
5 0.247119791527 0.752880208473 0.290382887441 0.709617112559
5 0.330644385105 0

In [10]:
# Save Resulst as CSV
dfResult = dfGameInfos.copy()

def PnScore(index, side):
    for res in res_epl_nb:
        if index == res[0]:
            if not math.isnan(res[side]):
                return res[side]
    return -1

dfResult['nb_pos_home'] = [PnScore(index, 1) for index in range(len(weeks))]
dfResult['nb_neg_home'] = [PnScore(index, 2) for index in range(len(weeks))]
dfResult['nb_pos_away'] = [PnScore(index, 3) for index in range(len(weeks))]
dfResult['nb_neg_away'] = [PnScore(index, 4) for index in range(len(weeks))]



# Save as CSV
useful_methods.DFtoCSV(dfResult, paths.READ_PATH_GAME_INFO, 'Predict-Review-63min(NB-HashEmolexData)', index=False)
print("[Saved in]: %s" % (paths.READ_PATH_GAME_INFO + 'Predict-Review-63min(NB-HashEmolexData)'))

[Saved in]: /Users/Bya/Dropbox/Research/datas/EPL/Predict-Review-63min(NB-HashEmolexData)


In [11]:
len(dfResult[dfResult['nb_neg_home'] != -1])

156

In [13]:
dfResult[
    ((dfResult.nb_pos_home > dfResult.nb_pos_away) & (dfResult.score_ft_home > dfResult.score_ft_away)) |
    ((dfResult.nb_pos_home < dfResult.nb_pos_away) & (dfResult.score_ft_home < dfResult.score_ft_away))
]

,GW,away_team,date,home_team,score_ft_away,score_ft_home,score_ht_away,score_ht_home,time,nb_pos_home,nb_neg_home,nb_pos_away,nb_neg_away
30,4,Arsenal,Sat. 29 Aug.,Newcastle,1,0,0,0,11:45,0.281951,0.718049,0.399604,0.600396
33,4,Crystal,Sat. 29 Aug.,Chelsea,2,1,0,0,14:00,0.430404,0.569596,0.592774,0.407226
36,4,Watford,Sat. 29 Aug.,City,0,2,0,0,14:00,0.501529,0.498471,0.393872,0.606128
38,4,Norwich,Sun. 30 Aug.,Southampton,0,3,0,1,12:30,0.390111,0.609889,0.344599,0.655401
39,4,United,Sun. 30 Aug.,Swansea,1,2,0,0,15:00,0.384990,0.615010,0.334014,0.665986
40,5,Chelsea,Sat. 12 Sep.,Everton,1,3,1,2,11:45,0.302150,0.697850,0.286535,0.713465
47,5,Tottenham,Sun. 13 Sep.,Sunderland,1,0,0,0,12:30,0.330644,0.669356,0.363505,0.636495
49,5,Newcastle,Mon. 14 Sep.,WestHam,0,2,0,1,19:00,0.332955,0.667045,0.229874,0.770126
52,6,Sunderland,Sat. 19 Sep.,Bournemouth,0,2,0,2,14:00,0.649346,0.350654,0.276535,0.723465
53,6,WestBromwich,Sat. 19 Sep.,Villa,1,0,1,0,14:00,0.259120,0.740880,0.287047,0.712953
